## Abstract
The song popularity dataset has multiple variable from which we can predict the song popularity. The main aim is to find which all variables are making most sense. And to evaluate the accuracy of the model using H2O Auto ML.Another aim is to find which all features has high multi-collienarity. To indentify how the song popularity is scored.The dataset seems cleaned already and has no null values in it. 

In [114]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip uninstall h2o
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Found existing installation: h2o 3.36.0.2
Uninstalling h2o-3.36.0.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h2o-3.36.0.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h2o/*
Proceed (y/n)? y
  Successfully uninstalled h2o-3.36.0.2
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
  Using cached h2o-3.36.0.2-py2.py3-none-any.whl


In [115]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
import matplotlib.pyplot as plt

In [116]:
# Set a minimum memory size and a run time in seconds
min_mem_size=6 
run_time=222

In [117]:
# Use 50% of availible resources
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

2


In [118]:
# 65535 Highest port no
# Start the H2O server on a random port
port_no=random.randint(5555,55555)

#  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
try:
  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
except:
  logging.critical('h2o.init')
  h2o.download_all_logs(dirname=logs_path, filename=logfile)      
  h2o.cluster().shutdown()
  sys.exit(2)

Checking whether there is an H2O instance running at http://localhost:47744 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprx93b6rt
  JVM stdout: /tmp/tmprx93b6rt/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprx93b6rt/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:47744
Connecting to H2O server at http://127.0.0.1:47744 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_fxbkv9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [119]:
url = "/content/drive/MyDrive/song_data.csv"
df = h2o.import_file(path = url)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [120]:
row=[15586]
df.drop(row,axis=0)


song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
Boulevard of Broken Dreams,73,262333,0.00552,0.496,0.682,2.94e-05,8,0.0589,-4.095,1,0.0294,167.06,4,0.474
In The End,66,216933,0.0103,0.542,0.853,0,3,0.108,-6.407,0,0.0498,105.256,4,0.37
Seven Nation Army,76,231733,0.00817,0.737,0.463,0.447,0,0.255,-7.828,1,0.0792,123.881,4,0.324
By The Way,74,216933,0.0264,0.451,0.97,0.00355,0,0.102,-4.938,1,0.107,122.444,4,0.198
How You Remind Me,56,223826,0.000954,0.447,0.766,0,10,0.113,-5.065,1,0.0313,172.011,4,0.574
Bring Me To Life,80,235893,0.00895,0.316,0.945,1.85e-06,4,0.396,-3.169,0,0.124,189.931,4,0.32
Last Resort,81,199893,0.000504,0.581,0.887,0.00111,4,0.268,-3.659,0,0.0624,90.578,4,0.724
Are You Gonna Be My Girl,76,213800,0.00148,0.613,0.953,0.000582,2,0.152,-3.435,1,0.0855,105.046,4,0.537
Mr. Brightside,80,222586,0.00108,0.33,0.936,0,1,0.0926,-3.66,1,0.0917,148.112,4,0.234
Sex on Fire,81,203346,0.00172,0.542,0.905,0.0104,9,0.136,-5.653,1,0.054,153.398,4,0.374


In [121]:
df['churn_bit'] = df['song_popularity'].asnumeric()
df.types
df.head()
df.describe()


Rows:18833
Cols:16




,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,churn_bit
type,enum,int,int,real,real,real,real,int,real,real,int,real,real,int,real,int
mins,,0.0,12000.0,1.02e-06,0.0,0.00107,0.0,0.0,0.0109,-38.768,0.0,0.0,0.0,0.0,0.0,0.0
mean,,52.99553974406624,218213.93394573362,0.25853325622895984,0.6333354165560454,0.6449665077258002,0.07800134884033351,5.289598045983115,0.17966188605108055,-7.447779164232999,0.62809961238252,0.1020927096054798,121.07396187543145,3.959114320607444,0.5279598099081398,52.99553974406624
maxs,,100.0,1799346.0,0.996,0.987,0.999,0.997,11.0,0.986,1.585,1.0,0.941,242.318,5.0,0.984,100.0
sigma,,21.903758102607654,59890.15209519732,0.2887172340963566,0.15672517264874103,0.214094569947863,0.22159666092674765,3.614542545488745,0.14398739991400542,3.827876542453302,0.4833248319923484,0.10437963825464319,28.71548387623507,0.29854842346733057,0.2446431184005412,21.903758102607654
zeros,,272,0,0,2,0,7150,2181,0,0,7004,2,2,3,2,272
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Boulevard of Broken Dreams,73.0,262333.0,0.005520000000000001,0.496,0.682,2.94e-05,8.0,0.0589,-4.095,1.0,0.0294,167.06,4.0,0.474,73.0
1,In The End,66.0,216933.0,0.0103,0.542,0.853,0.0,3.0,0.108,-6.407,0.0,0.0498,105.256,4.0,0.37,66.0
2,Seven Nation Army,76.0,231733.0,0.00817,0.737,0.463,0.447,0.0,0.255,-7.827999999999999,1.0,0.0792,123.881,4.0,0.324,76.0


In [122]:
df.shape

(18833, 16)

In [ ]:
pct_rows=0.80
df_train, df_test = df.split_frame([pct_rows])

In [ ]:
print(df_train.shape)
print(df_test.shape)


In [ ]:
df_train.head()

In [ ]:
# Set the features and target
X=df.columns
print(X)

In [ ]:
# Set target and predictor variables
y ='song_popularity'
y_numeric ='churn_bit'
X.remove(y)
X.remove(y_numeric) 
print(X)

In [ ]:
aml = H2OAutoML(max_runtime_secs=run_time, seed=1)

In [ ]:
aml.train(x=X,y=y,training_frame=df_train)      

In [ ]:
print(aml.leaderboard)

In [ ]:
model_index=0
glm_index=0
glm_model=''
aml_leaderboard_df=aml.leaderboard.as_data_frame()
models_dict={}
for m in aml_leaderboard_df['model_id']:
  models_dict[m]=model_index
  if 'StackedEnsemble' not in m:
    break 
  model_index=model_index+1  

for m in aml_leaderboard_df['model_id']:
  if 'GLM' in m:
    models_dict[m]=glm_index
    break  
  glm_index=glm_index+1     
models_dict

In [ ]:
print(model_index)
best_model = h2o.get_model(aml.leaderboard[model_index,'model_id'])

In [ ]:
best_model.algo

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category = matplotlib.cbook.mplDeprecation)

In [ ]:
print(best_model.rmse(train = True))

In [ ]:
print(best_model.rmse(train = False))

In [ ]:
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d

In [ ]:
mod_perf=best_model.model_performance(df_test)
stats_test={}
stats_test=model_performance_stats(mod_perf)
stats_test

In [ ]:
predictions = best_model.predict(df_test)

In [ ]:
best_model.model_performance(train=True)

In [ ]:
y_pred=h2o.as_list(predictions)
y_pred[0:100]

In [ ]:
y_test=h2o.as_list(df_test[y_numeric])
y_test[0:100]

In [ ]:
print(X)

In [ ]:
best_model.partial_plot(df, cols=['song_duration_ms'])

In [ ]:
 best_model.partial_plot(df, cols=['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode', 'speechiness', 'tempo', 'time_signature', 'audio_valence'])

In [ ]:
def shannon_entropy(p):   
    return (-p *np.log2(p) - (1-p)*np.log2(1-p))
base=0.0000000001
x = np.arange(base, 1.0-base, 0.01)
plt.figure(1)
plt.plot(x, shannon_entropy(x), 'go', x, shannon_entropy(x), 'k')
plt.ylabel('Shannon entropy(X)')
plt.xlabel('X')
plt.show()

###1) Is the relationship significant?


In [ ]:
if best_model.algo in ['gbm','drf','xrt','xgboost']:
  best_model.varimp_plot()

### Are any model assumptions violated?



There are four assumptions associated with a linear regression model:

1.   Linearity: The relationship between X and the mean of Y is linear.
2.   Homoscedasticity: The variance of residual is the same for any value of X
3.   Independence: Observations are independent of each other.
4.   Normality: For any fixed value of X, Y is normally distributed.



Data is linear because MSE value is too high. The data is linear and this is satisfied linearity assumption.
Ref : https://github.com/h2oai/h2o-tutorials/issues/50

In [ ]:
best_model.model_performance()

In [ ]:
best_model.plot()

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
import seaborn as sns
df_dp=pd.read_csv("/content/drive/MyDrive/song_data.csv")

def diagnostic_plots(df, variable):
    
    plt.figure(figsize=(20, 9))

    plt.subplot(1, 3, 1)
    sns.histplot(data = df_dp, x=variable, bins=30, kde=True)
    plt.title('Histogram')
    
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')
    
    plt.show()

In [ ]:
num_columns=df_dp[['song_duration_ms','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','audio_mode','speechiness','tempo','time_signature','audio_valence']].columns
num_columns

In [ ]:
for i in num_columns:
    diagnostic_plots(df_dp,i)

Normalization : Data is not normally distributed hence one of the model assumption is violated.

In [ ]:
import seaborn as sns
df_norm=pd.read_csv("/content/drive/MyDrive/song_data.csv")
df_norm=df_norm.iloc[:,1:-1]
ax=sns.distplot(df_norm)

### Is there any multi collinearity in the model?

As per the [Analytics Vidya](https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/) multi collinearity is something which occurs when more than two independent variables are highly correlated to each other. 

**Why is it an issue?**<br>
Multicollinearity can be a problem in a regression model because we would not be able to distinguish between the individual effects of the independent variables on the dependent variable. It will might **not effect effieciency** of the model but might effect the reiabibility of the model

**How to fix?**<br>
Highest multi collinear columns must be dropped to fix the multi collinearity issue

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# VIF dataframe
dfp=pd.read_csv("/content/drive/MyDrive/song_data.csv")

def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)


In [ ]:
X = dfp.iloc[:,2:-1]
calc_vif(X)

In [ ]:
preds = aml.predict(df_test)


In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
metalearner = h2o.get_model(se.metalearner()['name'])


In [ ]:
metalearner.coef_norm()

### In the multivariate models are predictor variables independent of all the other predictor variables?



You can tell if two random variables are independent by looking at their individual probabilities. If those probabilities don’t change when the events meet, then those variables are independent. Another way of saying this is that if the two variables are correlated, then they are not independent.[[Ref]](https://www.statisticshowto.com/independent-random-variables/#:~:text=You%20can%20tell%20if%20two,then%20they%20are%20not%20independent). So yes the variables are NOT independent in multivariate models. 

In [ ]:
df_n=pd.read_csv("/content/drive/MyDrive/song_data.csv")
df_n=df_n.iloc[:,2:-1]
cor_h2o=df_n.corr(method='pearson',min_periods=1).round(2)
sns.heatmap(cor_h2o, annot=True)
plt.show()

### In multivariate models rank the most significant predictor variables and exclude insignificant ones from the model.



Multiple Variable models have more than one variable which can act as predictor, as per [Elseiver](https://www.elsevier.es/en-revista-allergologia-et-immunopathologia-105-articulo-simple-linear-multivariate-regression-models-S0301054611001054). 

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

model_glm=H2OGeneralizedLinearEstimator(family = 'poisson', lambda_ = 0,
                                             remove_collinear_columns = True,
                                             compute_p_values = True)



In [ ]:
predictors=['song_duration_ms','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','audio_mode','speechiness','tempo','time_signature','audio_valence']

In [ ]:
response='song_popularity'

In [ ]:
model_glm.train(x = predictors, y = response, training_frame = df_train, validation_frame = df_test)

In [ ]:
varimp = model_glm.varimp(use_pandas=True)

In [ ]:
print(varimp)

In [ ]:
plt.figure(figsize=(20,15))

plt.plot(varimp['variable'],varimp['relative_importance'])


### Does the model make sense?



As per the model summary R^2 value is 0.7 for the model which is referred as high accuracy of the model with the help of H2o Auto ML

###Does regularization help?



Yes regularization helps alot MSE value before regularization was 110 and after it was 455 hence regularization is making sense to model

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
charges_glm = H2OGeneralizedLinearEstimator(family = 'gaussian', lambda_ = 0, compute_p_values = True)
charges_glm_regularization = H2OGeneralizedLinearEstimator(family = 'gaussian', lambda_ = .001, alpha = 0)

In [ ]:
df_r= df.drop(['song_popularity', 'song_name'], axis = 1)
X1=df_r.columns


In [ ]:
pct_rows=0.80
df1_train,df1_test = df.split_frame([pct_rows])

In [ ]:
charges_glm_regularization.train(x = X, y= y,training_frame = df1_train)

### Which independent variables are significant? 


In [ ]:
df_n=pd.read_csv("/content/drive/MyDrive/song_data.csv")

df_n=df_n.iloc[:,2:-1]
cor_h2o=df_n.corr(method='pearson',min_periods=1).round(2)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(cor_h2o, annot=True)
plt.show()

Filtered values more than 0.7 to find most relevant values from the below heatmap it shows energy and loudness are most correlated variables in the dataset. If we filter it less than 0.6 it will show relation between acousticness and energy

In [ ]:
cor_h2o = cor_h2o.unstack()
cor_h2o = cor_h2o[abs(cor_h2o) >= 0.7]
cor_h2o.describe()
sns.heatmap(cor_h2o, annot=True)
plt.show()

###Which hyperparameters are important?

Hyperparameters like gamma,max depth, split datasets will be hyperparameters which are important. 




### *Conclusion*
Completed the notebook here with the end notes as the auto ml library H2O.ai is working perfectly with the dataset and with effieicnt accuracy of the model. The notebook concludes that there are h2o library can help giving better accuracy with less amount of work. But still I believe the dataset is supposed to manipulated and made neccessary functions which library limits. The dateset was small but can be more accurate with more data. H2o library gave the results with accuracy of 77%. Notebook has considered with and without regularization effects to the model. Also which variables are important and directly dependent to each other.

## Licensing and Citation H2o.ai
H2O.ai (Feb 2022). Auto ML  for H2O, Python module version 3.10.0.8. <br>
Copyright (c) 2022 Author Saad Ghojaria.

